In [1]:
import torch 
from pbb.eval import *
from pbb.models import *
from pbb.bounds import PBBobj
from pbb import data

/Users/nsq277/opt/anaconda3/envs/pbb_tight/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
name_data = "mnist"
train, test = data.loaddataset(name_data)

In [3]:
device = "cpu"
loader_kargs = (
    {"num_workers": 1, "pin_memory": True} if torch.cuda.is_available() else {}
)

In [4]:
(
    _,
    train_1batch_loader,
    _,
    train1_1batch_loader,
    _,
    train2_1batch_loader,
    _,
    _,
) = data.loadbatches(
    train,
    test,
    loader_kargs,
    batch_size=250,
    prior=True,
    perc_train=1.0,
    perc_prior=0.5,
)

In [5]:
for data, target in train1_1batch_loader:
    input_S_1, target_S_1 = data.to(device), target.to(device)
for data, target in train2_1batch_loader:
    input_S_2, target_S_2 = data.to(device), target.to(device)
for data, target in train_1batch_loader:
    input_S, target_S = data.to(device), target.to(device)

In [ ]:
net0_mnist_fquad_learnt_1.0.pt

In [7]:
net0 = torch.load(
    "./saved_models/net0_mnist_fquad_learnt_1.0.pt",
    map_location=torch.device(device),
)
mu_net0 = get_mu(net0)

In [9]:
net1 = torch.load(
    "./saved_models/pnet1_mnist_fquad_learnt_1.0.pt",
    map_location=torch.device(device),
)
mu_pnet1 = get_mu_pnet(net1)
sigma_pnet1 = get_sigma_pnet(net1)

In [28]:
sigma_pnet1.sum()

tensor(35942.3516)

In [11]:
net2 = torch.load(
    "./saved_models/net2_mnist_fquad_learnt_1.0.pt",
    map_location=torch.device(device),
)
mu_pnet2 = get_mu_pnet(net2)
sigma_pnet2 = get_sigma_pnet(net2)

In [27]:
sigma_pnet2.sum()

tensor(35937.8945)

In [14]:
kl_approach3=get_kl_q_p(mu_pnet1, sigma_pnet1, mu_net0, 0.03)
kl_approach3

1971.5587158203125

In [15]:
get_kl_q_p(mu_pnet2, sigma_pnet2, mu_pnet1, sigma_pnet1)


133.07272338867188

In [ ]:
net1.eval()
get_loss_01(net1, input_S_2, target_S_2, sample=False).sum()

In [19]:
net2.eval()
get_loss_01(net2, input_S_2, target_S_2, sample=False).sum()

tensor(4107)

In [71]:
aa.shape

torch.Size([30000])

In [30]:
F.softmax(ouput_S_2, dim=1)[0,:][-2]

tensor(0.3354, grad_fn=<SelectBackward>)

In [87]:
(F.nll_loss(ouput_S_2, target_S_2, reduce=False) - aa)

tensor([1.2372, 1.8765, 1.3514,  ..., 0.9656, 1.7233, 1.7015],
       grad_fn=<SubBackward0>)

In [85]:
F.nll_loss(ouput_S_2, target_S_2) - aa.float().mean()

tensor(1.3796, grad_fn=<SubBackward0>)

In [93]:
F.sigmoid((F.nll_loss(ouput_S_2, target_S_2, reduce=False) - aa))

/Users/nsq277/opt/anaconda3/envs/pbb_tight/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/Users/nsq277/opt/anaconda3/envs/pbb_tight/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([0.7751, 0.8672, 0.7944,  ..., 0.7242, 0.8486, 0.8457],
       grad_fn=<SigmoidBackward>)

In [77]:
F.softmax((bb-1/2)).mean()

/var/folders/xf/5w99rbbs1g79w78bqgk29z2m0000gp/T/ipykernel_77051/2248117067.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax((bb-1/2)).mean()


tensor(3.3333e-05, grad_fn=<MeanBackward0>)

In [65]:
target_S_2

tensor([9, 0, 7,  ..., 4, 2, 3])

In [34]:
import torch
import torch.nn.functional as F

# Example ground truth probabilities (target) and predicted logits
target_probs = torch.tensor([0.1, 0.3, 0.6])
predicted_logits = torch.tensor([2.0, 1.0, 0.1])

# Apply softmax to get predicted probabilities
predicted_probs = F.softmax(predicted_logits, dim=0)

# Compute cross-entropy loss
cross_entropy_loss = F.cross_entropy(predicted_logits.view(1, -1), torch.tensor([torch.argmax(target_probs)]))

print("True probabilities:", target_probs)
print("Predicted probabilities:", predicted_probs)
print("Cross-entropy loss:", cross_entropy_loss.item())

True probabilities: tensor([0.1000, 0.3000, 0.6000])
Predicted probabilities: tensor([0.6590, 0.2424, 0.0986])
Cross-entropy loss: 2.3170299530029297


In [39]:
import torch.nn.functional as F

# Example ground truth probabilities (target) and predicted logits
target_probs = torch.tensor([0.1, 0.3, 0.6])
predicted_logits = torch.tensor([2.0, 1.0, 0.1])

# Apply softmax to get predicted probabilities
predicted_probs = F.softmax(predicted_logits, dim=0)

# Compute cross-entropy loss without summing over data points
cross_entropy_loss = F.cross_entropy(predicted_logits.view(1, -1), torch.tensor([torch.argmax(target_probs)]), reduce=False)

print("True probabilities:", target_probs)
print("Predicted probabilities:", predicted_probs)
print("Cross-entropy loss for each data point:", cross_entropy_loss)

True probabilities: tensor([0.1000, 0.3000, 0.6000])
Predicted probabilities: tensor([0.6590, 0.2424, 0.0986])
Cross-entropy loss for each data point: tensor([2.3170])


/Users/nsq277/opt/anaconda3/envs/pbb_tight/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [41]:
cross_entropy_loss

tensor([2.3170])

In [140]:
net1.state_dict()['l1.weight.mu'].shape

torch.Size([600, 784])

In [142]:
net0.state_dict()['l1.weight'].shape

torch.Size([600, 784])

In [155]:
net2.init_net

ModuleAttributeError: 'ProbNNet4l' object has no attribute 'init_net'

In [154]:
net1.l1.weight.rho.shape

torch.Size([600, 784])

In [152]:
net0.l1.weight.shape

torch.Size([600, 784])

In [20]:
net1.l1.weight.device="cpu"
net1.l1.bias.device="cpu"
net1.l1.weight_prior.device="cpu"
net1.l1.bias_prior.device="cpu"

net1.l2.weight.device="cpu"
net1.l2.bias.device="cpu"
net1.l2.weight_prior.device="cpu"
net1.l2.bias_prior.device="cpu"

net1.l3.weight.device="cpu"
net1.l3.bias.device="cpu"
net1.l3.weight_prior.device="cpu"
net1.l3.bias_prior.device="cpu"

net1.l4.weight.device="cpu"
net1.l4.bias.device="cpu"
net1.l4.weight_prior.device="cpu"
net1.l4.bias_prior.device="cpu"

In [21]:
net2.l1.weight.device="cpu"
net2.l1.bias.device="cpu"
net2.l1.weight_prior.device="cpu"
net2.l1.bias_prior.device="cpu"

net2.l2.weight.device="cpu"
net2.l2.bias.device="cpu"
net2.l2.weight_prior.device="cpu"
net2.l2.bias_prior.device="cpu"

net2.l3.weight.device="cpu"
net2.l3.bias.device="cpu"
net2.l3.weight_prior.device="cpu"
net2.l3.bias_prior.device="cpu"

net2.l4.weight.device="cpu"
net2.l4.bias.device="cpu"
net2.l4.weight_prior.device="cpu"
net2.l4.bias_prior.device="cpu"

In [22]:
bound = PBBobj(
        objective="fquad",
        pmin=1e-5,
        classes=10,
        delta=0.025,
        delta_test=0.01,
        mc_samples=10,
        kl_penalty=1.0,
        device="cpu",
        n_posterior=1,
        n_bound=len(input_S_2),
    )

In [23]:
train_obj, risk_ce, risk_01, kl, loss_ce_train, loss_01_train = (
                computeRiskCertificates(
                    net=net2,
                    toolarge=False,
                    pbobj=bound,
                    device=device,
                    lambda_var=1.0,
                    train_loader=train2_1batch_loader,
                    whole_train=train2_1batch_loader,
                )
            )
risk_01

0.7667577241770327

In [24]:
compute_final_stats_risk_delta(
    net2,
    net1,
    10,
    10,
    input_S,
    target_S,
    input_S_2,
    target_S_2,
    None,
    10,
    clamping=True,
    pmin=1e-5,
    delta_test=0.01,
)

100%|██████████| 10/10 [03:36<00:00, 21.62s/it]


0.3784678480132313

In [120]:
compute_final_stats_risk_delta(
    net2,
    net1,
    10,
    10,
    input_S,
    target_S,
    input_S_2,
    target_S_2,
    None,
    10,
    clamping=True,
    pmin=1e-5,
    delta_test=0.01,
)

100%|██████████| 10/10 [03:23<00:00, 20.33s/it]


0.6111762871697817

In [21]:
net1.eval()
get_loss_01(net1, input_S, target_S, sample=False, clamping=True, pmin=1e-5).sum()

tensor(11371)

In [22]:
net2.eval()
get_loss_01(net2, input_S, target_S, sample=False, clamping=True, pmin=1e-5).sum()

tensor(9391)

In [39]:
net1.eval()
get_loss_01(net1, input_S_2, target_S_2, sample=False, clamping=True, pmin=1e-5).sum()

tensor(619)

In [1]:
net2.l1

NameError: name 'net2' is not defined